In [1]:
import gym
import random
import numpy as np
#import time
from gym.envs.registration import register
from IPython.display import clear_output

In [10]:
enviorments = {
    'CartPole-v1': gym.make('CartPole-v1'),
    'MountainCar-v0': gym.make('MountainCar-v0'),
    'MountainCarContinuous-v0': gym.make('MountainCarContinuous-v0'),
    'Pendulum-v1': gym.make('Pendulum-v1'),
    'Acrobot-v1': gym.make('Acrobot-v1'),
    'FrozenLake-v1': gym.make('FrozenLake-v1', is_slippery=False)
}

/home/rodrigo/Documents/ML/venv/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


In [3]:
class Agent():
  def __init__(self, env):

    self.is_discrete = type(env.action_space) == gym.spaces.discrete.Discrete

    if self.is_discrete:
      self.action_size = env.action_space.n

    else:
      self.action_low = env.action_space.low
      self.action_high = env.action_space.high
      self.action_shape = env.action_space.shape
  
  def get_action(self, state):

    if self.is_discrete:
      #Acao randomica
      action = random.choice(range(self.action_size))

      #Acao com base na inclinacao do pendulo
      # pole_angle = state[2]
      # action = 0 if pole_angle < 0 else 1

    else:
      action = np.random.uniform(self.action_low, self.action_high, self.action_shape)

    return action

In [4]:
class QAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.01):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print("State size:", self.state_size)
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.build_model()
        
    def build_model(self):
        self.q_table = 1e-4*np.random.random([self.state_size, self.action_size])
        
    def get_action(self, state):
        q_state = self.q_table[state]
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience):
        state, action, next_state, reward, done = experience
        
        q_next = self.q_table[next_state]
        q_next = np.zeros([self.action_size]) if done else q_next
        q_target = reward + self.discount_rate * np.max(q_next)
        
        q_update = q_target - self.q_table[state,action]
        self.q_table[state,action] += self.learning_rate * q_update
        
        if done:
            self.eps = self.eps * 0.99

In [6]:
env = enviorments['CartPole-v1']
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

agent = Agent(env)
state = env.reset()

for _ in range(200):
    action = agent.get_action(state)
    state, reward, done, info = env.step(action)
    env.render()

Observation space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
Action space: Discrete(2)


/home/rodrigo/Documents/ML/venv/lib/python3.8/site-packages/gym/envs/classic_control/cartpole.py:168: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  logger.warn(


In [11]:
env = enviorments['FrozenLake-v1']
agent = QAgent(env)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

State size: 16
Observation space: Discrete(16)
Action space: Discrete(4)


In [13]:
total_reward = 0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state,action,next_state,reward,done))
        state = next_state
        total_reward += reward
        
        print("s:", state, "a:", action)
        print("Episode: {}, Total reward: {}, eps: {}".format(ep,total_reward,agent.eps))
        env.render()
        #print(agent.q_table)
        clear_output(wait=True)

s: 5 a: 2
Episode: 99, Total reward: 3.0, eps: 0.13397967485796175
